# Introduction au traitement d'images avec NumPy

Ce chapitre introduit les concepts fondamentaux du traitement d'images en Python. Nous explorerons comment les images sont représentées sous forme de tableaux NumPy, comment analyser leurs propriétés, et comment effectuer des opérations de base comme la visualisation des canaux RGB, la conversion en niveaux de gris, l'ajout de bruit et le floutage.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np  # Pour la manipulation de tableaux
from skimage import data  # Pour charger des images d'exemple
import matplotlib.pyplot as plt  # Pour la visualisation

---

## 1. Propriétés de base d'une image RGB

Une image numérique est représentée comme un tableau NumPy à 3 dimensions :
-   **Dimensions** : `(hauteur, largeur, canaux)`
-   **Canaux RGB** : Rouge (R), Vert (G), Bleu (B)
-   **Valeurs** : Généralement des entiers de 0 à 255 (uint8)

Nous allons charger une image d'exemple et examiner ses attributs statistiques globaux ainsi que pour chaque canal.

In [ ]:
# Chargement d'une image d'exemple (astronaute)
# Les images RGB réelles ont 3 canaux de couleur
img_arr = data.astronaut()

# Affichage des propriétés de l'image
print('shape/type', img_arr.shape, img_arr.dtype)
print('min/max/mean/std', np.min(img_arr), np.max(img_arr), np.average(img_arr), np.std(img_arr))

# Statistiques pour chaque canal de couleur
# Canal 0 = Rouge, Canal 1 = Vert, Canal 2 = Bleu
print('mean/std R', np.average(img_arr[:,:,0]), np.std(img_arr[:,:,0]))
print('mean/std G', np.average(img_arr[:,:,1]), np.std(img_arr[:,:,1]))
print('mean/std B', np.average(img_arr[:,:,2]), np.std(img_arr[:,:,2]))

---

## 2. Visualisation des canaux RGB individuels

Pour mieux comprendre la composition d'une image couleur, nous pouvons visualiser chaque canal séparément :
-   **Image originale** : Les trois canaux RGB combinés
-   **Canal Rouge** : Conserve uniquement le canal R (indices 0), met G et B à zéro
-   **Canal Vert** : Conserve uniquement le canal G (indice 1), met R et B à zéro
-   **Canal Bleu** : Conserve uniquement le canal B (indice 2), met R et G à zéro

Cette technique permet d'identifier quelle couleur contribue le plus à différentes régions de l'image.

In [ ]:
# Création d'une figure avec 4 sous-graphiques
fig, axs = plt.subplots(1, 4, figsize=(16, 4))

# Affichage de l'image originale
im = axs[0].imshow(img_arr)

# Affichage du canal Rouge uniquement
# On met les canaux 1 et 2 (Vert et Bleu) à zéro
tmp_arr = img_arr.copy()
tmp_arr[:,:,1:3] = 0
im = axs[1].imshow(tmp_arr)

# Affichage du canal Vert uniquement
# On met les canaux 0 et 2 (Rouge et Bleu) à zéro
tmp_arr = img_arr.copy()
tmp_arr[:,:,0:3:2] = 0
im = axs[2].imshow(tmp_arr)

# Affichage du canal Bleu uniquement
# On met les canaux 0 et 1 (Rouge et Vert) à zéro
tmp_arr = img_arr.copy()
tmp_arr[:,:,0:2] = 0
im = axs[3].imshow(tmp_arr)

fig.tight_layout()
plt.show()

---

## 3. Conversion en niveaux de gris

Les valeurs RGB sont simplement des nombres. On peut les traiter de différentes façons :
-   **Moyenne des canaux** : Convertit une image RGB en niveaux de gris en calculant la moyenne des trois canaux
-   **Visualisation en colormap 'gray'** : Affiche les valeurs d'intensité en niveaux de gris

Cette approche simplifie l'image tout en conservant l'information d'intensité lumineuse.

In [ ]:
# Conversion RGB vers niveaux de gris par moyenne
fig, axs = plt.subplots(1, 4, figsize=(16, 4))

# Calcul de la moyenne des trois canaux RGB
# axis=-1 signifie "le long du dernier axe" (les canaux)
gray_arr = np.average(img_arr, axis=-1)

# Ajout du canal gris comme 4e canal pour visualisation
concat_arr = np.append(img_arr, np.expand_dims(gray_arr,axis=-1), axis=-1)

# Affichage des 3 canaux RGB plus le canal gris
for i, ax in enumerate(axs):
    # cmap='gray' : affiche les valeurs en niveaux de gris
    im = ax.imshow(concat_arr[:,:, i], cmap='gray')

fig.tight_layout()
plt.show()

---

## 4. Ajout de bruit à une image

Le bruit est une perturbation aléatoire qui dégrade la qualité d'une image :
-   **Bruit aléatoire** : Généré avec `np.random.random()` et multiplié par un facteur
-   **Corrélation** : Mesure la similarité entre l'image originale et l'image bruitée
-   **Plus de bruit** : Plus la corrélation diminue (s'approche de 0)

Cet exercice montre comment le bruit affecte progressivement la qualité visuelle et statistique d'une image.

In [ ]:
# Ajout de niveaux croissants de bruit à l'image
fig, axs = plt.subplots(1, 4, figsize=(16, 4))

# Utilisation de l'image en niveaux de gris
gray_arr = np.average(img_arr, axis=-1)
print(np.prod(gray_arr.shape))

# Pour chaque sous-graphique, ajouter un niveau de bruit différent
for i, ax in enumerate(axs):
    # Génération de bruit aléatoire avec amplitude croissante (100*i)
    noise = np.random.random(262144).reshape(gray_arr.shape)*(100*i)
    
    # Affichage de l'image bruitée
    im = ax.imshow(gray_arr+noise, cmap='gray')
    
    # Calcul du coefficient de corrélation entre l'image originale et bruitée
    # .ravel() : aplatit le tableau 2D en 1D pour le calcul de corrélation
    correlation_coefficient = np.round(np.corrcoef(gray_arr.ravel(), (gray_arr+noise).ravel())[0,1], 4)
    print(i, 'correlation', correlation_coefficient)

fig.tight_layout()
plt.show()

---

## 5. Floutage et comparaison d'images

Le filtre gaussien est une opération de convolution courante pour lisser les images :
-   **`gaussian_filter()`** : Applique un flou gaussien avec un paramètre `sigma` (écart-type)
-   **Plus sigma est grand** : Plus le flou est prononcé
-   **Soustraction** : La différence entre l'image originale et l'image floue révèle les détails haute fréquence

Cette technique est utile pour la réduction de bruit ou pour isoler les contours et détails fins.

In [ ]:
# Application d'un filtre gaussien pour flouter l'image
from scipy.ndimage import gaussian_filter

fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Image originale en niveaux de gris
im = axs[0].imshow(gray_arr, cmap='gray')

# Application du flou gaussien avec sigma=2
# sigma contrôle l'intensité du flou (plus sigma est grand, plus le flou est fort)
tmp_arr = gaussian_filter(gray_arr, sigma=2)
print(tmp_arr.shape)
im = axs[1].imshow(tmp_arr, cmap='gray')

# Différence entre l'image originale et l'image floue
# Révèle les détails haute fréquence (contours, textures)
im = axs[2].imshow(gray_arr - tmp_arr, cmap='gray')

fig.tight_layout()
plt.show()